# 전처리

## 데이터 불러오기

판례목록, 판례본문

In [1]:
import pandas as pd
import numpy as np
import re

df_search = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\lawSearch_yk.csv')
df_service = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\lawService_yk.csv')

### 판례본문 전처리

판례본문 복사

In [2]:
df_service_split = df_service.copy()

판례내용 분리 (판례내용_상단, 판례내용_이유)

In [3]:
# 판례본문 내용 분리 - 이유 기준 분리

# '【이' 앞부분과 '【이'를 포함한 뒷부분으로 나누는 코드
df_service_split['판례내용_상단'] = df_service['판례내용'].str.split('【이').str[0]
df_service_split['판례내용_이유'] = df_service['판례내용'].str.extract(r'(【이.*)')

# 결과 출력
df_service_split.head()

,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유
0,147393,2008다71575,선고,판결,[1] 국가 또는 상위 지방자치단체가 법령의 규정 등에 의하여 그 권한의 일부를 하...,[1] 국가 또는 상위 지방자치단체가 법령의 규정 등에 의하여 그 권한의 일부를 하...,"[1] \r\n 민법 제194조, \r\n 지방자치법 제9...",[1][2][3] \r\n 대법원 2010. 10. 14. 선고 200...,"【원고, 피상고인】 망 \r\n <br/>【피고, 상고인】 대한민국...","【원고, 피상고인】 망 \r\n <br/>【피고, 상고인】 대한민국...","【이 유】 상고이유에 대하여 판단한다.<br/> 1. 원심은, 현재의 서울..."
1,147384,2008두10133,선고,판결,피상속인이 제3자를 위하여 연대보증채무를 부담하고 있었지만 상속개시 당시에는 아직 ...,구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것) 제45조...,구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것) 제45조...,NaN,"【원고, 상고인】 <br/>【피고, 피상고인】 북대구세무서장<br/>【원심판결】 대...","【원고, 상고인】 <br/>【피고, 피상고인】 북대구세무서장<br/>【원심판결】 대...",【이 유】 상고이유를 본다.<br/> 1. 상속세 경정청구 부분에 관하여<...
2,150031,2010도891,선고,판결,[1] 착오로 송금되어 입금된 돈을 임의로 인출하여 소비한 행위가 송금인과 피고인 ...,NaN,[1] \r\n 형법 제355조 제1항 / [2] \r\n ...,[1] \r\n 대법원 1968. 7. 24. 선고 1966도1705 ...,【피 고 인】 <br/>【상 고 인】 검사<br/>【변 호 인】 변호사 이승욱<br...,【피 고 인】 <br/>【상 고 인】 검사<br/>【변 호 인】 변호사 이승욱<br...,【이 유】 상고이유를 본다. <br/>어떤 예금계좌에 돈이 착오로 잘못 송금...
3,188185,2010누12639,선고,판결,NaN,NaN,NaN,NaN,"【원고, 항소인】 전국공무원노동조합 (소송대리인 법무법인 시민 담당변호사 전영식)<...","【원고, 항소인】 전국공무원노동조합 (소송대리인 법무법인 시민 담당변호사 전영식)<...",【이 유】1. 제1심 판결의 인용<br/> 이 법원이 이 사건에 관하여 설시...
4,160470,2010가합8730,선고,일부판결,NaN,NaN,NaN,NaN,【원 고】 진해동부신용협동조합 (소송대리인 법무법인 미래로 담당변호사 김형석)...,【원 고】 진해동부신용협동조합 (소송대리인 법무법인 미래로 담당변호사 김형석)...,【이 유】1. 기초사실<br/> 아래 각 사실은 당사자 사이에 다툼이 없거나...


판례내용_상단(재판당사자 추출)

In [5]:
# 판례내용 상단에서 재판당사자 추출

# '문단의 첫 번째 '【'부터 '【주' 앞까지의 텍스트를 포함하여 추출
def extract_text_with_first_bracket_to_ju(content):
    # 정규식으로 첫 번째 '【'부터 '【주'까지 텍스트 추출 (첫 번째 '【' 포함)
    match = re.search(r'(【.*?【\s*주)', content, re.DOTALL)
    if match:
        return match.group(1).strip()  # 추출한 텍스트의 양쪽 공백 제거
    return None  # 매치되지 않으면 None 반환

# '재판당사자' 열 생성 (첫 번째 '【'부터 '【주'까지 추출, 첫 번째 '【' 포함)
df_service_split['재판당사자'] = df_service_split['판례내용_상단'].apply(extract_text_with_first_bracket_to_ju)

# 결과 확인
#df_service_split.head()

def extract_and_clean_text(content):
    if isinstance(content, str):  # 내용이 문자열인지 확인
        matches = re.findall(r'【(.*?)】', content)  # '【'와 '】' 사이의 모든 텍스트 찾기
        cleaned_texts = [match.replace(' ', '') for match in matches]  # 공백 제거
        return cleaned_texts  # 결과 리스트 반환
    return []  # 문자열이 아닌 경우 빈 리스트 반환

# 결과 열 생성
df_service_split['재판당사자_텍스트'] = df_service_split['재판당사자'].apply(extract_and_clean_text)

# 결과 확인
df_service_split[['재판당사자_텍스트']]

,재판당사자_텍스트
0,"[원고,피상고인, 피고,상고인, 원심판결]"
1,"[원고,상고인, 피고,피상고인, 원심판결]"
2,"[피고인, 상고인, 변호인, 원심판결]"
3,"[원고,항소인, 피고,피항소인, 제1심판결, 변론종결]"
4,"[원고, 피고, 변론종결]"
...,...
22495,"[피고인, 상고인, 원심판결]"
22496,"[원고,상고인, 피고,피상고인, 피고보조참가인, 원심판결]"
22497,"[원고,상고인, 피고,피상고인, 원심판결]"
22498,"[원고,상고인, 피고,피상고인, 원심판결]"


판례내용_이유

In [6]:
# 판례내용 이유 부분의 대괄호를 삭제하여 본문으로 변환

import re
import pandas as pd

service_reason = []

for idx in range(len(df_service_split)):
    content = df_service_split['판례내용_이유'].iloc[idx]
    
    # 문자열인지 확인 (문자열이 아니면 건너뜀)
    if isinstance(content, str):
        content_parts = re.split(r'【(.*?)】', content)
        reason_dict = {}

        for j in range(len(content_parts)):
            if content_parts[j].replace(' ', '') in ['이유']:
                cleaned_content = content_parts[j+1]
                cleaned_content = cleaned_content.replace('<br/>', '').replace('\r', '').replace('\n', '')
                cleaned_content = cleaned_content.replace('【', '').replace('】', '').strip()
                cleaned_content = re.sub(r'\s+', ' ', cleaned_content)
                reason_dict[content_parts[j].replace(' ', '')] = cleaned_content
        
        service_reason.append(reason_dict)
    else:
        # 비문자열 데이터 처리 (필요한 경우 빈 딕셔너리 또는 다른 기본값)
        service_reason.append({})

# 결과를 '판례내용_이유2' 열에 저장
df_service_split['판례내용_이유2'] = service_reason

In [8]:
df_service_split['판례내용_이유2'].iloc[:5]

0     {'이유': '상고이유에 대하여 판단한다. 1. 원심은, 현재의 서울 강남구 세곡동'}
1    {'이유': '상고이유를 본다. 1. 상속세 경정청구 부분에 관하여 상고법원은 상고...
2    {'이유': '상고이유를 본다. 어떤 예금계좌에 돈이 착오로 잘못 송금되어 입금된 ...
3    {'이유': '1. 제1심 판결의 인용 이 법원이 이 사건에 관하여 설시할 이유는,...
4    {'이유': '1. 기초사실 아래 각 사실은 당사자 사이에 다툼이 없거나, 갑 제1...
Name: 판례내용_이유2, dtype: object

# 데이터 통합

### 판례목록, 판례본문 통합

In [9]:
# 중복삭제: 판례내용(df_service) 판례정보일련번호, 사건번호 열삭제, 열제거거 

df_service_pre=df_service_split.drop(columns=['판례정보일련번호','사건번호'])

In [10]:
# 데이터프레임 통합, 결합

df_combined = pd.concat([df_search,df_service_pre], axis=1)
df_combined.head()

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,재판당사자,재판당사자_텍스트,판례내용_이유2
0,147393,소유권보존등기말소,2008다71575,2010.12.09,대법원,민사,/DRF/lawService.do?OC=ykdb0508&target=prec&ID=...,선고,판결,[1] 국가 또는 상위 지방자치단체가 법령의 규정 등에 의하여 그 권한의 일부를 하...,[1] 국가 또는 상위 지방자치단체가 법령의 규정 등에 의하여 그 권한의 일부를 하...,"[1] \r\n 민법 제194조, \r\n 지방자치법 제9...",[1][2][3] \r\n 대법원 2010. 10. 14. 선고 200...,"【원고, 피상고인】 망 \r\n <br/>【피고, 상고인】 대한민국...","【원고, 피상고인】 망 \r\n <br/>【피고, 상고인】 대한민국...","【이 유】 상고이유에 대하여 판단한다.<br/> 1. 원심은, 현재의 서울...","【원고, 피상고인】 망 \r\n <br/>【피고, 상고인】 대한민국...","[원고,피상고인, 피고,상고인, 원심판결]","{'이유': '상고이유에 대하여 판단한다. 1. 원심은, 현재의 서울 강남구 세곡동'}"
1,147384,상속세경정청구,2008두10133,2010.12.09,대법원,세무,/DRF/lawService.do?OC=ykdb0508&target=prec&ID=...,선고,판결,피상속인이 제3자를 위하여 연대보증채무를 부담하고 있었지만 상속개시 당시에는 아직 ...,구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것) 제45조...,구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것) 제45조...,NaN,"【원고, 상고인】 <br/>【피고, 피상고인】 북대구세무서장<br/>【원심판결】 대...","【원고, 상고인】 <br/>【피고, 피상고인】 북대구세무서장<br/>【원심판결】 대...",【이 유】 상고이유를 본다.<br/> 1. 상속세 경정청구 부분에 관하여<...,"【원고, 상고인】 <br/>【피고, 피상고인】 북대구세무서장<br/>【원심판결】 대...","[원고,상고인, 피고,피상고인, 원심판결]",{'이유': '상고이유를 본다. 1. 상속세 경정청구 부분에 관하여 상고법원은 상고...
2,150031,횡령(인정된죄명:점유이탈물횡령),2010도891,2010.12.09,대법원,형사,/DRF/lawService.do?OC=ykdb0508&target=prec&ID=...,선고,판결,[1] 착오로 송금되어 입금된 돈을 임의로 인출하여 소비한 행위가 송금인과 피고인 ...,NaN,[1] \r\n 형법 제355조 제1항 / [2] \r\n ...,[1] \r\n 대법원 1968. 7. 24. 선고 1966도1705 ...,【피 고 인】 <br/>【상 고 인】 검사<br/>【변 호 인】 변호사 이승욱<br...,【피 고 인】 <br/>【상 고 인】 검사<br/>【변 호 인】 변호사 이승욱<br...,【이 유】 상고이유를 본다. <br/>어떤 예금계좌에 돈이 착오로 잘못 송금...,【피 고 인】 <br/>【상 고 인】 검사<br/>【변 호 인】 변호사 이승욱<br...,"[피고인, 상고인, 변호인, 원심판결]",{'이유': '상고이유를 본다. 어떤 예금계좌에 돈이 착오로 잘못 송금되어 입금된 ...
3,188185,통보처분취소,2010누12639,2010.12.08,서울고등법원,일반행정,/DRF/lawService.do?OC=ykdb0508&target=prec&ID=...,선고,판결,NaN,NaN,NaN,NaN,"【원고, 항소인】 전국공무원노동조합 (소송대리인 법무법인 시민 담당변호사 전영식)<...","【원고, 항소인】 전국공무원노동조합 (소송대리인 법무법인 시민 담당변호사 전영식)<...",【이 유】1. 제1심 판결의 인용<br/> 이 법원이 이 사건에 관하여 설시...,"【원고, 항소인】 전국공무원노동조합 (소송대리인 법무법인 시민 담당변호사 전영식)<...","[원고,항소인, 피고,피항소인, 제1심판결, 변론종결]","{'이유': '1. 제1심 판결의 인용 이 법원이 이 사건에 관하여 설시할 이유는,..."
4,160470,배당 이의,2010가합8730,2010.12.08,창원지방법원,민사,/DRF/lawService.do?OC=ykdb0508&target=prec&ID=...,선고,일부판결,NaN,NaN,NaN,NaN,【원 고】 진해동부신용협동조합 (소송대리인 법무법인 미래로 담당변호사 김형석)...,【원 고】 진해동부신용협동조합 (소송대리인 법무법인 미래로 담당변호사 김형석)...,【이 유】1. 기초사실<br/> 아래 각 사실은 당사자 사이에 다툼이 없거나...,【원 고】 진해동부신용협동조합 (소송대리인 법무법인 미래로 담당변호사 김형석)...,"[원고, 피고, 변론종결]","{'이유': '1. 기초사실 아래 각 사실은 당사자 사이에 다툼이 없거나, 갑 제1..."


In [11]:
df_combined.to_csv('Labor_datacombied.csv', index=False)